In [50]:
from requests import get
from io import BytesIO
from PyPDF2 import PdfFileReader
from tabula import read_pdf
from datetime import date
import pandas as pd
import numpy as np
import math

In [2]:
url = "https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf"
response = get(url)

In [51]:
remote_file = response.content
memory_file = BytesIO(remote_file)
pdf = read_pdf(memory_file, pages=1)

df1 = pdf[0]
df1.head()

,.,Unnamed: 0,Total Cases
0,Total,NaN,26697
1,Median Age (Range),NaN,47 (0-105)
2,Age Group,NaN,NaN
3,- 0 to 17,NaN,543 (2%)
4,- 18 to 44,NaN,11617 (44%)


In [52]:
today = date.today()

df2 = df1.rename(columns={'.': 'Variable', 'Total Cases': today})
df2 = df2.drop(columns=['Unnamed: 0'])

df2.head()

,Variable,2020-03-27
0,Total,26697
1,Median Age (Range),47 (0-105)
2,Age Group,NaN
3,- 0 to 17,543 (2%)
4,- 18 to 44,11617 (44%)


In [53]:
df3 = df2.copy()

df3.insert(0, "Category", None)

category = None
for index, row in df3.iterrows():
    split_var = row['Variable'].split("-  ")
    if len(split_var) > 1:
        row['Variable'] = split_var[1]
    if type(row[today]) == str:
        split_tot = row[today].split(" (")
        if len(split_tot) > 1:
            row[today] = split_tot[0]
    elif math.isnan(row[today]):
        category = row['Variable']
    if row['Variable'] != 'Deaths':
        row['Category'] = category

for index, row in df3.iterrows():
    if row['Category'] == row['Variable']:
        df3.drop(index, inplace=True)
        
df3 = df3.reset_index(drop=True)
            
df3

,Category,Variable,2020-03-27
0,None,Total,26697
1,None,Median Age (Range),47
2,Age Group,0 to 17,543
3,Age Group,18 to 44,11617
4,Age Group,45 to 64,9158
5,Age Group,65 to 74,3034
6,Age Group,75 and over,2286
7,Age Group,Unknown,59
8,Age 50 and over,Yes,12209
9,Age 50 and over,No,14429


In [54]:
file_name = "covid_df.csv"
covid_hist = pd.read_csv(file_name, index_col=0)

covid_hist.head()

,Category,Variable,2020-03-26,2020-03-27
0,NaN,Total,23112,26697
1,NaN,Median Age (Range),47,47
2,Age Group,0 to 17,495,543
3,Age Group,18 to 44,10145,11617
4,Age Group,45 to 64,7869,9158


In [55]:
print(covid_hist.columns[-1])

2020-03-27


In [56]:
df4 = df3.copy()
if covid_hist.columns[-1] == str(today):
    covid_hist = covid_hist.drop(columns=[str(today)])
df5 = covid_hist.join(df4[today])

df5.head()

,Category,Variable,2020-03-26,2020-03-27
0,NaN,Total,23112,26697
1,NaN,Median Age (Range),47,47
2,Age Group,0 to 17,495,543
3,Age Group,18 to 44,10145,11617
4,Age Group,45 to 64,7869,9158


In [57]:
covid_df = df5.copy()

covid_df.to_csv(file_name)

backup_name = "covid_df_" + str(today) + ".csv"
backup_path = "Daily CSVs/"
covid_df.to_csv(backup_path + backup_name, date_format='%Y-%m-%d')